In [2]:
# allow import of modules from src/
import sys
sys.path.append("../src")

from pathlib import Path

from pandas import DataFrame

import yappi

from Lifting import *
from parsing.Parser import parse_gm
import Solver

import logging

In [3]:
games_dir = Path("../material/test_cases/")

liftings = {"Input": InputLifting,
            "Random (seed 42)": lambda game: RandomLifting(game, 42),
            "Successor": SuccessorLifting,
            "Self loop": SelfLoopLifting}

results_path = Path("../results/test_cases/2023-01-09_11-26.csv")

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

In [4]:
game_files = [file for file in games_dir.iterdir() if file.suffix == '.gm']

games = {}
for game_file in game_files:
    games[game_file.name] = parse_gm(game_file)

In [7]:
statistics = []

for game_name, game in games.items():
    for lifting_name, lifting in liftings.items():
        logging.info("Profiling game {game_name} with lifting {lifting_name}.".format(game_name=game_name, lifting_name=lifting_name))

        # lifting

        # initialize yappi
        yappi.clear_stats()
        yappi.start()

        lift = lifting(game)

        # stop yappi
        yappi.stop()

        lifting_ttot = yappi.get_thread_stats()[0].ttot

        # solving

        # initialize yappi
        yappi.clear_stats()
        yappi.start()

        result, solver_iterations = Solver.solve_game(game, lift)

        # stop yappi
        yappi.stop()

        solver_ttot = yappi.get_thread_stats()[0].ttot

        statistics.append((game_name, lifting_name, result, lifting_ttot, solver_iterations, solver_ttot))

11:32:36:INFO:Profiling game test_case_all_flipped.gm with lifting Input.
11:32:36:INFO:Profiling game test_case_all_flipped.gm with lifting Random (seed 42).
11:32:36:INFO:Profiling game test_case_all_flipped.gm with lifting Successor.
11:32:36:INFO:Profiling game test_case_all_flipped.gm with lifting Self loop.
11:32:36:INFO:Profiling game test_case_all_odd.gm with lifting Input.
11:32:36:INFO:Profiling game test_case_all_odd.gm with lifting Random (seed 42).
11:32:36:INFO:Profiling game test_case_all_odd.gm with lifting Successor.
11:32:36:INFO:Profiling game test_case_all_odd.gm with lifting Self loop.
11:32:36:INFO:Profiling game test_case_different_order.gm with lifting Input.
11:32:36:INFO:Profiling game test_case_different_order.gm with lifting Random (seed 42).
11:32:36:INFO:Profiling game test_case_different_order.gm with lifting Successor.
11:32:36:INFO:Profiling game test_case_different_order.gm with lifting Self loop.
11:32:36:INFO:Profiling game test_case_no_self_loop.gm 

In [8]:
df = DataFrame(statistics, columns = ["Game", "Lifting", "Result", "Lifting ttot", "Solver iterations", "Solver ttot"])
df

,Game,Lifting,Result,Lifting ttot,Solver iterations,Solver ttot
0,test_case_all_flipped.gm,Input,"{'even': {0, 1, 2, 3}, 'odd': {}}",0.000101,4,0.000785
1,test_case_all_flipped.gm,Random (seed 42),"{'even': {0, 1, 2, 3}, 'odd': {}}",0.000055,4,0.001371
2,test_case_all_flipped.gm,Successor,"{'even': {0, 1, 2, 3}, 'odd': {}}",0.000071,4,0.000840
3,test_case_all_flipped.gm,Self loop,"{'even': {0, 1, 2, 3}, 'odd': {}}",0.000065,4,0.000517
4,test_case_all_odd.gm,Input,"{'even': {}, 'odd': {0, 1, 2, 3}}",0.000037,6,0.000729
5,test_case_all_odd.gm,Random (seed 42),"{'even': {}, 'odd': {0, 1, 2, 3}}",0.000067,5,0.001306
6,test_case_all_odd.gm,Successor,"{'even': {}, 'odd': {0, 1, 2, 3}}",0.000069,6,0.001172
7,test_case_all_odd.gm,Self loop,"{'even': {}, 'odd': {0, 1, 2, 3}}",0.000074,5,0.000570
8,test_case_different_order.gm,Input,"{'even': {4, 5, 6}, 'odd': {0, 1, 2, 3}}",0.000032,8,0.001636
9,test_case_different_order.gm,Random (seed 42),"{'even': {4, 5, 6}, 'odd': {0, 1, 2, 3}}",0.000081,6,0.004550


In [ ]:
# Save data frame
df.to_csv(results_path)